<a href="https://colab.research.google.com/github/trikebike/airpg/blob/main/rpg2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Character, Item, and Location classes:

In [16]:
import json

class Character:
    """A character in the game."""

    def __init__(self, name, character_class, location):
        """Initialize the character.

        Args:
            name: The character's name.
            character_class: The character's class.
            location: The character's starting location.
        """
        self.name = name
        self.character_class = character_class
        self.location = location
        self.health = 100
        self.strength = 10
        self.intelligence = 10
        self.inventory = []

    def move(self, location):
        """Move the character to a new location.

        Args:
            location: The new location.
        """
        self.location = location
        print(f"You moved to {location.name}.")

    def use_item(self, item_name):
        """Use an item.

        Args:
            item_name: The name of the item to use.
        """
        for i, item in enumerate(self.inventory):
            if item.name.lower() == item_name.lower():
                item.use(self)
                del self.inventory[i]
                return
        print("You don't have that item.")

    def gain_experience(self, amount):
        """Gain experience.

        Args:
            amount: The amount of experience to gain.
        """
        # TODO: Increase character's level or stats when they gain enough experience.
        pass

    def add_item(self, item):
        """Add an item to the character's inventory.

        Args:
            item: The item to add.
        """
        self.inventory.append(item)
        print(f"You received item: {item.name}.")

class Item:
    """An item in the game."""

    def __init__(self, name, item_type, power):
        """Initialize the item.

        Args:
            name: The item's name.
            item_type: The item's type.
            power: The item's power.
        """
        self.name = name
        self.item_type = item_type
        self.power = power

    def use(self, character):
        """Use the item on the character.

        Args:
            character: The character to use the item on.
        """
        if self.item_type == "healing":
            character.health += self.power
            print(f"You healed {self.power} points!")
        # TODO: Add more item types as needed.

class Location:
    """A location in the game."""

    def __init__(self, name, description, connections):
        """Initialize the location.

        Args:
            name: The location's name.
            description: The location's description.
            connections: The locations that are connected to this location.
        """
        self.name = name
        self.description = description
        self.connections = connections

Monster and Quest classes:

In [18]:
class Monster:
    """A monster in the game."""

    def __init__(self, name, health, strength, intelligence):
        """Initialize the monster.

        Args:
            name: The monster's name.
            health: The monster's health.
            strength: The monster's strength.
            intelligence: The monster's intelligence.
        """
        self.name = name
        self.health = health
        self.strength = strength
        self.intelligence = intelligence

    def attack(self, character):
        """Attack the character.

        Args:
            character: The character to attack.
        """
        character.take_damage(self.strength)
        print(f"{self.name} attacks {character.name} for {self.strength} damage!")

    def is_dead(self):
        """Return True if the monster is dead.

        Returns:
            True if the monster is dead, False otherwise.
        """
        return self.health <= 0

class Quest:
    """A quest in the game."""

    def __init__(self, description, completion_condition, experience_reward, item_reward=None):
        """Initialize the quest.

        Args:
            description: The quest's description.
            completion_condition: The condition that must be met to complete the quest.
            experience_reward: The amount of experience to reward the player when they complete the quest.
            item_reward: The item to reward the player when they complete the quest.
        """
        self.description = description
        self.completion_condition = completion_condition
        self.experience_reward = experience_reward
        self.item_reward = item_reward
        self.completed = False

    def check_completion(self, character):
        """Check if the quest is completed.

        Args:
            character: The character to check the quest for.

        Returns:
            True if the quest is completed, False otherwise.
        """
        if not self.completed and self.completion_condition(character):
            print(f"You completed the quest: {self.description}!")
            character.gain_experience(self.experience_reward)
            if self.item_reward:
                character.add_item(self.item_reward)
            self.completed = True

Game class:

In [24]:
class Game:
    def __init__(self, character):
        self.character = character
        self.running = False

    def start(self):
        self.running = True
        while self.running:
            command = input("> ")
            self.process_command(command)

    def process_command(self, command):
        command_parts = command.split(" ")
        if command_parts[0] == "go":
            destination_name = " ".join(command_parts[1:])
            for location in self.character.location.connections:
                if location.name.lower() == destination_name.lower():
                    self.character.move(location)
                    return
            print("You can't go there.")
        elif command_parts[0] == "use":
            item_name = " ".join(command_parts[1:])
            self.character.use_item(item_name)
        elif command_parts[0] == "save":
            self.save()
        elif command_parts[0] == "load":
            self.load()
        elif command_parts[0] == "quit":
            self.stop()
        else:
            print("Invalid command.")

    def save(self):
        with open('savegame.json', 'w') as savegame:
            json.dump({
                'health': self.character.health,
                'strength': self.character.strength,
                'intelligence': self.character.intelligence,
                'inventory': [(item.name, item.item_type, item.power) for item in self.character.inventory]
                # add more character attributes as needed
            }, savegame)

    def load(self):
        try:
            with open('savegame.json', 'r') as savegame:
                save_data = json.load(savegame)
            self.character.health = save_data['health']
            self.character.strength = save_data['strength']
            self.character.intelligence = save_data['intelligence']
            self.character.inventory = [Item(name, item_type, power) for name, item_type, power in save_data['inventory']]
            print("Game loaded successfully.")
        except FileNotFoundError:
            print("No saved game found.")

    def stop(self):
        self.running = False


Setup and starting the game:

In [19]:
class Game:
    """A game."""

    def __init__(self, character):
        """Initialize the game.

        Args:
            character: The character to start the game with.
        """
        self.character = character
        self.running = False

    def start(self):
        """Start the game.

        """
        self.running = True
        while self.running:
            command = input("> ")
            self.process_command(command)

    def process_command(self, command):
        """Process a command.

        Args:
            command: The command to process.
        """
        command_parts = command.split(" ")
        if command_parts[0] == "go":
            destination_name = " ".join(command_parts[1:])
            for location in self.character.location.connections:
                if location.name.lower() == destination_name.lower():
                    self.character.move(location)
                    return
            print("You can't go there.")
        elif command_parts[0] == "use":
            item_name = " ".join(command_parts[1:])
            self.character.use_item(item_name)
        elif command_parts[0] == "save":
            self.save()
        elif command_parts[0] == "load":
            self.load()
        elif command_parts[0] == "quit":
            self.stop()
        else:
            print("Invalid command.")

    def save(self):
        """Save the game.

        """
        with open('savegame.json', 'w') as savegame:
            json.dump({
                'character': self.character.__dict__,
                'location': self.character.location.__dict__,
            }, savegame)

    def load(self):
        """Load the game.

        """
        try:
            with open('savegame.json', 'r') as savegame:
                save_data = json.load(savegame)
            self.character.__dict__.update(save_data['character'])
            self.character.location.__dict__.update(save_data['location'])
            print("Game loaded successfully.")
        except FileNotFoundError:
            print("No saved game found.")

    def stop(self):
        """Stop the game.

        """
        self.running = False


Graphical Interface

Textual Interface